In [12]:
!pip install  --quiet langchain-google-genai
!pip install --quiet langchain

In [21]:
import time
from google.colab import userdata
from typing import List, Dict

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [22]:
google_api_key = userdata.get('GOOGLE_API_KEY')

model: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model="gemini-1.5-flash",
                                                      api_key=google_api_key)

model.invoke("What is capital of Australia?").content

'The capital of Australia is **Canberra**. \n'

In [23]:
class Agent:
  def __init__(self, name: str, role: str, skills: List[str]):
    self.name = name
    self.role = role
    self.skills = skills
    self.model = model

  def process(self, task: str, context: List[Dict] = None):
    messages = [
        SystemMessage(content=f"""You are {self.name}, a {self.role} with following
        skills: {", ".join(self.skills)}. Respond to the surgeon query based
        on your role and skills"""),
    ]
    if context: #check if context available
      for message in context: # for every message in context
        if message["role"] == "human":
          messages.append(HumanMessage(content=message["content"]))
        elif message["role"] == "ai":
          messages.append(AIMessage(content=message["content"]))
    # if no context, the first message is a human message whose content is the
    # the task that is assigned to the agent
    messages.append(HumanMessage(content=task))
    # process the messages with llm
    response = self.model.invoke(messages)
    return response.content

In [33]:
# defininig specialized agents
class AnatomySpecialistAgent(Agent):
  def __init__(self):
    super().__init__("James",
                     "Human Anatomy Specialist",
                      ["deep knowledge of human anatomy", "understanding of surgery complexities", "suggesting suitable actions for surgical procedures in context of anatomy"]
                    )

class SurgicalRiskAgent(Agent):
  def __init__(self):
    super().__init__("Alice",
                     "Surgical Risk Analyst",
                     ["analyses risks of surgeries", "surgical guidance", "complication prevention"]
                    )

In [41]:
# defining different functions for collaboration system

def anatomy_specialist_function(anatomy_agent, task: str, context: list):
  print("Anatomy Agent: Analyzing anatomy aspects of surgery")
  anatomy_task = f"""Provide related anatomy insights to the surgeon query {task} that
                may help in successful surgery
                """
  result = anatomy_agent.process(anatomy_task)
  context.append({"role": "ai", "content": f"{result}"})
  print(f"Anatomy Agent: {result[:150]}... \n")
  return context


def risk_analyst_function(risk_agent, task: str, context: list):
  print("Risk Analyst: Analyzing surgical risks")
  risk_task = f"""Provide risk insighted related to the tast {task} that may help
              in successful surgery
              """
  result = risk_agent.process(risk_task)
  context.append({"role": "ai", "content": f"{result}"})
  print(f"Risk Analyst: {result[:150]}... \n")
  return context

def identify_organs_function(anatomy_agent, task: str, context:list):
  print("Anatomy Agent: Identifying organs involved in surgery")
  anatomy_task = f"""Identify related organs involved in the surgery {task} that
                may help related to the surgery
                """
  result = anatomy_agent.process(anatomy_task)
  context.append({"role": "ai", "content": f"{result}"})
  print(f"Anatomy Agent: {result[:150]}... \n")
  return context

def risk_assessment_function(risk_agent, task: str, context: list):
  print("Risk Analyst: Assessing surgical risks")
  risk_task = f"""Provide risk insighted related to the task {task} and the related
              context {context} that may help in successful surgery. Your response
              should be brief and conversational like you are answering a question
              """
  result = risk_agent.process(risk_task)
  context.append({"role": "ai", "content": f"{result}"})
  print(f"Risk Analyst: {result[:150]}... \n")
  return context


In [42]:
class SurgicalAssistantSystem:
  def __init__(self):
    self.anatomy_agent = AnatomySpecialistAgent()
    self.risk_agent = SurgicalRiskAgent()

  def solve(self, task: str, timeout: int = 400):
    print(f"\n Starting collaboration to solve {task}\n")
    start_time = time.time()
    context = []
    steps = [
        (anatomy_specialist_function, self.anatomy_agent),
        (risk_analyst_function, self.risk_agent),
        (identify_organs_function, self.anatomy_agent),
        (risk_assessment_function, self.risk_agent),
        (anatomy_specialist_function, self.anatomy_agent),
        (risk_assessment_function, self.risk_agent),
    ]
    for step_func, agent in steps:
      if time.time() - start_time > timeout:
        return "Operation timed out"
      try:
        result = step_func(agent, task, context)
        if isinstance(result, str):
          return result
      except Exception as e:
        return f"Error during collaboration: {str(e)}"

    print("\n Collaboration completed successfully!")
    return context[-1]['content']

In [ ]:
# prompt: generate an example conversation between surgeon and assistant surgeon during a cardiac surgery. The assistant surgeon is performing surgery and surgeon is supervising. The assistant surgeon ask question and the suregon responds precisely. Store it in a variable conversation that is of type list

conversation = [
    {"speaker": "Assistant Surgeon", "utterance": "Surgeon, could you please confirm the location of the left anterior descending artery?"},
    {"speaker": "Surgeon", "utterance": "Yes, it's located on the front surface of the heart, running along the left ventricle, supplying blood to the front part of the heart muscle."},
    {"speaker": "Assistant Surgeon", "utterance": "Okay. I'm about to cannulate the superior vena cava. Any specific tips for this step?"},
    {"speaker": "Surgeon", "utterance": "Ensure you avoid any damage to the surrounding structures, especially the right atrium. A gentle and controlled approach is crucial."},
    {"speaker": "Assistant Surgeon", "utterance": "I'm encountering some bleeding from a small vessel near the right atrium. How should I manage it?"},
    {"speaker": "Surgeon", "utterance": "Use a bipolar cautery to seal the bleeding vessel. Be cautious not to damage adjacent tissues."},
    {"speaker": "Assistant Surgeon", "utterance": "I'm getting ready to place the aortic cross clamp. Should I start with a gentle clamping force?"},
    {"speaker": "Surgeon", "utterance": "Yes, start with a gentle force and progressively increase to ensure a complete occlusion. Monitor the aortic pressure carefully."},
    {"speaker": "Assistant Surgeon", "utterance": "I'm noticing some slight ventricular fibrillation during the cross-clamp procedure. What do we do?"},
    {"speaker": "Surgeon", "utterance": "Let's begin cardioplegic arrest immediately. Ensure proper delivery of the cardioplegic solution into the aortic root."},
    {"speaker": "Assistant Surgeon", "utterance": "The coronary ostia appear to be slightly obscured. Can you guide me on identifying them clearly?"},
    {"speaker": "Surgeon", "utterance": "Use a gentle saline flush to clear any blood clots and enhance visualization. Look for the openings located at the sinus of Valsalva."},
    {"speaker": "Assistant Surgeon", "utterance": "Surgeon, I've initiated the bypass circuit and now need to manage the patient's temperature during hypothermia. What are your preferences?"},
    {"speaker": "Surgeon", "utterance": "Let's maintain the core body temperature around 28°C. Use a combination of surface cooling and cooled irrigation solutions."},
    {"speaker": "Assistant Surgeon", "utterance": "The mitral valve is severely calcified. How should I approach its repair?"},
    {"speaker": "Surgeon", "utterance": "Let's proceed with a cautious approach and examine the leaflet mobility before trying to excise any calcium deposits. Use a sharp debridement technique if needed."},
    {"speaker": "Assistant Surgeon", "utterance": "The bypass graft is ready to be anastomosed. Could you please review the placement?"},
    {"speaker": "Surgeon", "utterance": "Yes, it's well positioned. Ensure the distal and proximal anastomoses are adequately secured."},
    {"speaker": "Assistant Surgeon", "utterance": "Surgeon, I'm about to remove the aortic cross clamp. What's the expected circulatory response?"},
    {"speaker": "Surgeon", "utterance": "Expect a gradual return of spontaneous circulation. Monitor the rhythm and blood pressure closely."},
    {"speaker": "Assistant Surgeon", "utterance": "We've achieved adequate reperfusion. Do you want me to monitor the patient's urine output now?"},
    {"speaker": "Surgeon", "utterance": "Yes, that's important. Let's closely monitor the patient's urine output and the kidney function parameters to ensure a successful outcome."}
]

In [43]:
surgical_ai = SurgicalAssistantSystem()
question = "Okay. I'm about to cannulate the superior vena cava. Any specific tips for this step?"
result = surgical_ai.solve(question)
print(result)


 Starting collaboration to solve Okay. I'm about to cannulate the superior vena cava. Any specific tips for this step?

Anatomy Agent: Analyzing anatomy aspects of surgery
Anatomy Agent: Okay, I can definitely provide some anatomical insights for cannulating the superior vena cava. Here are some key points to consider:

**1. Anatomical... 

Risk Analyst: Analyzing surgical risks
Risk Analyst: Okay, I understand you're about to cannulate the superior vena cava.  Here are some risk insights and tips to help ensure a successful procedure:

**R... 

Anatomy Agent: Identifying organs involved in surgery
Anatomy Agent: Okay, I understand. Cannulating the superior vena cava is a crucial step in many surgical procedures. Here are some anatomical considerations and tips... 

Risk Analyst: Assessing surgical risks
Risk Analyst: ... 

Anatomy Agent: Analyzing anatomy aspects of surgery
Anatomy Agent: Okay, I can provide some anatomical insights to help with cannulating the superior vena cava. 

